In [33]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

url = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/Medicoes_2018-2024.csv"

df = pd.read_csv(url,  sep=',', encoding='latin-1', skiprows=1)

df['Potência Ativa'] = pd.to_numeric(df['Potência Ativa'].str.replace(',', '.'), errors='coerce')
df['Potência Reativa'] = pd.to_numeric(df['Potência Reativa'].str.replace(',', '.'), errors='coerce')

# df.tail(3)


In [34]:
df

,Equipamento Medição,Data/Hora Medição,Potência Ativa,Potência Reativa
0,ABR_DJ_12B1,01/01/2018 00:00:00,9.90,0.00
1,ABR_DJ_12B1,01/01/2018 01:00:00,9.20,0.00
2,ABR_DJ_12B1,01/01/2018 02:00:00,8.90,0.00
3,ABR_DJ_12B1,01/01/2018 03:00:00,8.80,0.00
4,ABR_DJ_12B1,01/01/2018 04:00:00,8.50,0.00
...,...,...,...,...
4969604,URN_DJ_12B1,30/12/2024 23:00:00,1.72,0.33
4969605,URN_DJ_12B1,31/12/2024 00:00:00,1.70,0.34
4969606,URN_DJ_12B1,31/12/2024 03:00:00,1.57,0.31
4969607,URN_DJ_12B1,31/12/2024 04:00:00,1.49,0.27


In [ ]:
# Filtrando pra um transformador específico
# df_ctr = df[df['TRAFO'] == 'CTR_DJ_12B1']

# Criar o boxplot para a potência ativa de CTR-02T1
# plt.figure(figsize=(10, 6))
# sns.boxplot(y='P', data=df_ctr)
# plt.title('Boxplot da Potência Ativa para o Transformador CTR-02T1')
# plt.ylabel('Potência Ativa')
# plt.show()

In [35]:
# Função para aplicar o filtro IQR
def filtrar_coluna_iqr(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 3.2 * IQR

    return df[(df[coluna] >= lower_bound) & (df[coluna] <= upper_bound)]

# Função para aplicar o filtro baseado em Média Móvel
def filtrar_coluna_media_movel(df, coluna, janela=20, threshold=4):

    media_movel = df[coluna].rolling(window=janela, center=True).mean()
    desvio = np.abs(df[coluna] - media_movel)

    limite_superior = media_movel + threshold * desvio.std()
    limite_inferior = media_movel - threshold * desvio.std()

    return df[(df[coluna] >= limite_inferior) & (df[coluna] <= limite_superior)]

# Função para aplicar o filtro baseado em Z-Score
def filtrar_coluna_zscore(df, coluna, threshold=3):
    media = df[coluna].mean()
    desvio_padrao = df[coluna].std()
    z_scores = (df[coluna] - media) / desvio_padrao

    return df[np.abs(z_scores) < threshold]

# Função que aplica o filtro escolhido
def aplicar_filtro(df, coluna, metodo='iqr', janela=20, threshold=4, z_threshold=3):
    if metodo == 'iqr':
        return filtrar_coluna_iqr(df, coluna)
    elif metodo == 'media_movel':
        return filtrar_coluna_media_movel(df, coluna, janela=janela, threshold=threshold)
    elif metodo == 'zscore':
        return filtrar_coluna_zscore(df, coluna, threshold=z_threshold)
    else:
        raise ValueError("Método inválido! Escolha entre 'iqr', 'media_movel' ou 'zscore'.")


In [39]:
df.rename(columns={
    'Potência Ativa': 'P',
    'Potência Reativa': 'Q',
    'Data/Hora Medição': 'datahora',
    'Equipamento Medição': 'TRAFO'
}, inplace=True)

# Convertendo para datetime e salvando o módulo dos valores, pois pode ocorrer valores negativos
# df['datahora'] = pd.to_datetime(df['datahora'], dayfirst=True, errors='coerce')
df['datahora'] = pd.to_datetime(df['datahora'], format='%d/%m/%Y %H:%M:%S')

df['P'] = df['P'].abs()  # salvando o módulo dos valores
df['Q'] = df['Q'].abs()
df['S'] = np.sqrt(df['P']**2 + df['Q']**2)  # Calculando a potência aparente (S), em kVA

# Removendo linhas com S = 0
df = df[df['S'] != 0]

# DataFrame final para armazenar os dados filtrados
df_filtrado = pd.DataFrame()

# Escolha do método de filtragem ('iqr', 'media_movel' ou 'zscore')
metodo_filtro = 'iqr'

# Aplicando o filtro pra limpar os outliers pra cada transformador
for trafo in df['TRAFO'].unique():
    df_trafo = df[df['TRAFO'] == trafo]

    df_trafo_filtrado = aplicar_filtro(df_trafo, 'S', metodo=metodo_filtro, janela=20, threshold=4, z_threshold=3)

    # Adicionar os dados filtrados ao DataFrame final
    df_filtrado = pd.concat([df_filtrado, df_trafo_filtrado], ignore_index=True)

# Ordenando e removendo duplicatas
df_filtrado = df_filtrado.sort_values(by=['TRAFO', 'datahora'])
df_filtrado = df_filtrado.drop_duplicates(subset=['datahora', 'TRAFO'])

In [ ]:
# # Criando gráfico da Potência Aparente ao longo do tempo, separada por TRAFO
# fig_aparente = px.line(df_filtrado, x='datahora', y='S', color='TRAFO',
#                        title='Potência Aparente ao Longo do Tempo por Transformador',
#                        labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

# # Exibir ou salvar o gráfico em
# fig_aparente.show()
# fig_aparente.write_html("Demanda ao longo do tempo - IQR.html")

In [41]:
df_daily = df_filtrado.groupby('TRAFO', group_keys=False).apply(lambda x: x.set_index('datahora').resample('D').max()).reset_index()

df_count = df_daily.groupby('TRAFO').count()
df_count.sort_values('datahora').tail(41)

lista_trafos = df_count['TRAFO'].unique().tolist()
lista_trafos
# trafos_escolhidos = lista_trafos

KeyError: 'TRAFO'

In [14]:
import pandas as pd

# Definir intervalo de datas completo
data_inicio = '2018-01-01'
data_fim = '2024-12-31'
datas_completas = pd.date_range(start=data_inicio, end=data_fim, freq='D')

# Escolher um transformador específico (substituir 'TRAFO_X' pelo nome desejado)
trafo_escolhido = 'SPE_DJ_12B2'
datas_presentes = df[df['TRAFO'] == trafo_escolhido]['datahora'].dt.date.unique()

# Converter para um conjunto de datas para facilitar a comparação
datas_faltantes = set(datas_completas.date) - set(datas_presentes)

# Mostrar os dias que estão faltando
datas_faltantes


{datetime.date(2020, 6, 4),
 datetime.date(2020, 6, 14),
 datetime.date(2020, 6, 29),
 datetime.date(2020, 7, 5),
 datetime.date(2020, 7, 24),
 datetime.date(2020, 7, 30),
 datetime.date(2020, 8, 4),
 datetime.date(2020, 8, 14),
 datetime.date(2020, 12, 29),
 datetime.date(2020, 12, 30),
 datetime.date(2021, 3, 13),
 datetime.date(2021, 3, 14),
 datetime.date(2021, 4, 11),
 datetime.date(2021, 5, 18),
 datetime.date(2021, 12, 28),
 datetime.date(2021, 12, 29),
 datetime.date(2021, 12, 30),
 datetime.date(2022, 1, 8),
 datetime.date(2022, 1, 9),
 datetime.date(2022, 1, 10),
 datetime.date(2022, 1, 11),
 datetime.date(2023, 10, 28),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 30),
 datetime.date(2023, 11, 14),
 datetime.date(2023, 11, 15),
 datetime.date(2023, 12, 1),
 datetime.date(2023, 12, 2),
 datetime.date(2023, 12, 3),
 datetime.date(2023, 12, 4),
 datetime.date(2023, 12, 5),
 datetime.date(2023, 12, 21),
 datetime.date(2023, 12, 22),
 datetime.date(2023, 12, 23),
 datet

In [15]:
import pandas as pd

# Criar intervalo de datas completo
data_inicio = '2018-01-01'
data_fim = '2024-12-31'
datas_completas = pd.date_range(start=data_inicio, end=data_fim, freq='D')

# Função para preencher dados ausentes com interpolação
def preencher_e_interpolar(grupo):
    grupo = grupo.set_index('datahora')  # Define 'datahora' como índice
    grupo = grupo.reindex(datas_completas)  # Reindexa para ter todas as datas
    grupo = grupo.interpolate(method='linear')  # Interpola valores faltantes
    grupo = grupo.ffill().bfill()  # Preenche NaNs restantes (caso extremo)
    grupo['TRAFO'] = grupo['TRAFO'].iloc[0]  # Reatribui o trafo
    return grupo.reset_index().rename(columns={'index': 'datahora'})

# Aplicar a função para cada transformador
df_preenchido = df.groupby('TRAFO', group_keys=False).apply(preencher_e_interpolar)

# Exibir resultado
print(df_preenchido.head())


    datahora        TRAFO     P    Q     S
0 2018-01-01  ABR_DJ_12B1 11.40 0.00 11.40
1 2018-01-02  ABR_DJ_12B1 13.90 0.00 13.90
2 2018-01-03  ABR_DJ_12B1 13.60 0.00 13.60
3 2018-01-04  ABR_DJ_12B1 14.00 0.00 14.00
4 2018-01-05  ABR_DJ_12B1 13.60 0.00 13.60


In [17]:
df_preenchido

,datahora,TRAFO,P,Q,S
0,2018-01-01,ABR_DJ_12B1,11.40,0.00,11.40
1,2018-01-02,ABR_DJ_12B1,13.90,0.00,13.90
2,2018-01-03,ABR_DJ_12B1,13.60,0.00,13.60
3,2018-01-04,ABR_DJ_12B1,14.00,0.00,14.00
4,2018-01-05,ABR_DJ_12B1,13.60,0.00,13.60
...,...,...,...,...,...
2552,2024-12-27,URN_DJ_12B1,158.04,39.54,160.69
2553,2024-12-28,URN_DJ_12B1,1.67,0.42,1.71
2554,2024-12-29,URN_DJ_12B1,1.68,0.38,1.71
2555,2024-12-30,URN_DJ_12B1,1.72,0.44,1.75


#### Caso precise, ST

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from sklearn.preprocessing import MinMaxScaler
# from math import sqrt
# import random

# # Fixando seeds para reprodutibilidade
# np.random.seed(42)
# tf.random.set_seed(42)
# random.seed(42)

# def plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo):
#     plt.figure(figsize=(14, 6))
#     plt.plot(df_previsoes, y_test_inverso, label='Valores Reais', color='blue')
#     plt.plot(df_previsoes, y_pred, label=f'Predictions {modelo} - Transformer {trafo}', linestyle='--', color='orange')
#     plt.xlabel('Date')
#     plt.ylabel('Apparent Power')
#     plt.title(f'Predicted x Real Comparison - Transformer {trafo} ({modelo})')
#     plt.legend()
#     plt.show()

# def treinar_e_prever_modelo(df_filtrado, trafos_escolhidos, modelo, janela, epochs=20, batch_size=32):
#     resultados = []

#     for trafo in trafos_escolhidos:
#         df = df_filtrado[df_filtrado['TRAFO'] == trafo]
#         df = df[['datahora', 'S']]
#         df = df.set_index(['datahora']).resample('D').max()
#         df.sort_index(inplace=True)
#         df['S'] = df['S'].interpolate(method='linear')

#         dados = df[['S']].values
#         scaler = MinMaxScaler(feature_range=(0, 1))
#         dados_normalizados = scaler.fit_transform(dados)

#         X, y = [], []
#         for i in range(janela, len(dados_normalizados)):
#             X.append(dados_normalizados[i-janela:i, 0])
#             y.append(dados_normalizados[i, 0])

#         X, y = np.array(X), np.array(y)
#         X = np.reshape(X, (X.shape[0], X.shape[1], 1)) if modelo == 'LSTM' else X

#         split = int(len(X) * 0.83)
#         X_train, X_test = X[:split], X[split:]
#         y_train, y_test = y[:split], y[split:]

#         if modelo == 'SVR':
#             regressor = SVR(kernel='rbf', C=100, gamma=0.001, epsilon=0.01)

#         elif modelo == 'RFR':
#             regressor = RandomForestRegressor(n_estimators=100, max_depth=20, max_features='sqrt', n_jobs=-1, random_state=42)
#         elif modelo == 'LSTM':
#             regressor = Sequential()
#             regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
#             regressor.add(LSTM(units=50))
#             regressor.add(Dense(1))
#             regressor.compile(optimizer='adam', loss='mean_squared_error')
#             regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

#         regressor.fit(X_train, y_train) if modelo != 'LSTM' else None
#         y_pred_normalizado = regressor.predict(X_test)
#         y_pred = scaler.inverse_transform(y_pred_normalizado.reshape(-1, 1))
#         y_test_inverso = scaler.inverse_transform(y_test.reshape(-1, 1))

#         mse = mean_squared_error(y_test_inverso, y_pred)
#         rmse = sqrt(mse)
#         mae = mean_absolute_error(y_test_inverso, y_pred)
#         r2 = r2_score(y_test_inverso, y_pred)

#         df_previsoes = df.index[split + janela:]
#         plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo)

#         resultados.append({
#             'Trafo': trafo,
#             'Modelo': modelo,
#             'RMSE': rmse,
#             'MAE': mae,
#             'R2': r2
#         })

#     return pd.DataFrame(resultados)

# # Chamando para SVR, RFR e LSTM
# resultados_svr = treinar_e_prever_modelo(df_filtrado, trafos_escolhidos, modelo='SVR', janela=365)
# resultados_rfr = treinar_e_prever_modelo(df_filtrado, trafos_escolhidos, modelo='RFR', janela=365)
# resultados_lstm = treinar_e_prever_modelo(df_filtrado, trafos_escolhidos, modelo='LSTM', janela=365, epochs=20, batch_size=32)

# resultados_svr.to_csv('SVR_S.csv', sep=';')
# resultados_rfr.to_csv('RFR_S.csv', sep=';')
# resultados_lstm.to_csv('LSTM_S.csv', sep=';')

# print("Resultados SVR:")
# print(resultados_svr)
# print("\nResultados RFR:")
# print(resultados_rfr)
# print("\nResultados LSTM:")
# print(resultados_lstm)
